In [10]:
#import
import os
import pickle
import io

import decimal
import collections
import numpy as np
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn import ensemble
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

cwd = os.getcwd()
print (cwd)

min_max_scaler = preprocessing.MinMaxScaler()
max_abs_scaler = preprocessing.MaxAbsScaler()


def float_to_str(f):
    """
    Convert the given float to a string,
    without resorting to scientific notation
    """
    d1 = ctx.create_decimal(repr(f))
    return format(d1, 'f')

def parse_labeled_file(filename):
    f = open(filename, encoding="utf8")
    array = []

    for line in f:
        parts = line.split('\t')
        parts[0] = int(parts[0])
        parts[len(parts) - 1] = parts[len(parts) - 1].replace('\n', '')
        parts[len(parts) - 1] = int(parts[len(parts) - 1])
        array.append(parts)

    return array

def load_pickle(filepath):
    documents_f = open(filepath, 'rb')
    file = pickle.load(documents_f)
    documents_f.close()

    return file

def loadTrainSet(path_features, path_labels):
    X = load_pickle(path_features)
    y = load_pickle(path_labels)
    return X, y

def loadTestSet(path_features):
    X = load_pickle(path_features)
    return X

def load_output(filename):
    with open(filename, 'r') as fread:
        lines = fread.readlines()
    return lines


# Max number of digits for the computed scores
ctx = decimal.Context()
ctx.prec = 5
    
#features_acro = ["N", "L", "C", "E", "NL", "NC", "NE", "LC"]
#features_acro = ["N", "L", "C", "E", "NL", "NC", "NE", "LC", "LE"]
features_acro = ["NW", "CW", "LW", "NC", "NL", "LC", "NCW", "NLW", "LCW", "NLC", "NLCW"]

#All_models = [['random_forest_balanced', True], ['random_forest', False], ['svc_rbf', True], ['svc_rbf', False], ['sgd_log', True], ['sgd_log', False], ['nn_lbfgs', True], ['nn_lbfgs', False], ['svc_linear', True], ['svc_linear', False]]
All_models = [['random_forest_balanced', 'True']]

model_output_path = '/projets/sig/mullah/nlp/checkthat18/output/test'
model_combined_output_path = model_output_path + '/combined_model_feature'

test_files_2018 = ["task1-en-file1.txt","task1-en-file2.txt","task1-en-file3.txt","task1-en-file4.txt",
                  "task1-en-file5.txt","task1-en-file6.txt","task1-en-file7.txt"]    

for filename in test_files_2018:   
    print ("Processing test file: {}".format(filename))

    for model in All_models:
        print ("ML model ...:{}".format(model))
        for feature in features_acro:
            print ("Combing models for features :{}".format(feature))
            scores = []
            for fdx in range(0, len(feature)):
                model_output_filename = model_output_path + '/' +feature[fdx] + '/' + filename[:-4] + '_' + model[0] + '.txt'
                print ("loading model: {}".format(model_output_filename))
                
                scores.append([])
                prediction = load_output(model_output_filename)   
                for line in prediction:
                    cline = line.rstrip().split('\t')
                    scores[fdx].append(float(cline[1]))
            
            output_file = open(model_combined_output_path + "/" + filename[:-4] + '_' + 
                               model[0] + '_' + feature + '_' + 'combined' + '.txt', 'w')        
            scores_sm = np.transpose(scores)
            scores_minmax = min_max_scaler.fit_transform(scores_sm)
            
            for i in range(len(scores_sm)):
                # score = min(scores[i])
                #score = max(scores_minmax[i])
                # score = np.median(np.transpose(scores)[i])
                #score = np.prod(scores_minmax[i])
                #score = np.mean(scores_minmax[i])
                #count = (scores_minmax[i]>=score).sum()
                #score = score * count
                #count = (scores_sm[i]>=.5).sum()
                #score = count
                #score = np.mean(scores_minmax[i])
                
                #score = np.mean(scores_sm[i])
                total = len(scores_sm[i])
                
                vote = (scores_minmax[i]>=.5).sum()
                if vote >= 1:
                    score = np.max(scores_minmax[i])
                    #score = np.sum(scores_minmax[i])
                else:
                    score = np.mean(scores_minmax[i])
                
                #count = (scores_sm[i]>=.5).sum()
                #if count >= (total/2):
                #    score = max(scores_sm[i])
                #else:
                #    score = np.mean(scores_sm[i])
                #if count >= 1:
                #    score = max(scores_sm[i])
                #else:
                #    score = np.mean(scores_sm[i])
                output_file.write(str(i + 1) + "\t" + float_to_str(score) + "\n")
            output_file.close()
            print ("combined.")
            

/users/sig/mullah/nlp/projects/checkthat/notebooks
Processing test file: task1-en-file1.txt
ML model ...:['random_forest_balanced', 'True']
Combing models for features :NW
loading model: /projets/sig/mullah/nlp/checkthat18/output/test/N/task1-en-file1_random_forest_balanced.txt
loading model: /projets/sig/mullah/nlp/checkthat18/output/test/W/task1-en-file1_random_forest_balanced.txt
combined.
Combing models for features :CW
loading model: /projets/sig/mullah/nlp/checkthat18/output/test/C/task1-en-file1_random_forest_balanced.txt
loading model: /projets/sig/mullah/nlp/checkthat18/output/test/W/task1-en-file1_random_forest_balanced.txt
combined.
Combing models for features :LW
loading model: /projets/sig/mullah/nlp/checkthat18/output/test/L/task1-en-file1_random_forest_balanced.txt
loading model: /projets/sig/mullah/nlp/checkthat18/output/test/W/task1-en-file1_random_forest_balanced.txt
combined.
Combing models for features :NC
loading model: /projets/sig/mullah/nlp/checkthat18/output/te